# Pump Sensor Predict Breakpoint - PreProcessing Strategy

https://towardsdatascience.com/lstm-autoencoder-for-extreme-rare-event-classification-in-keras-ce209a224cfb

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
PATH_DATA = "../data/sensor_cleaned.csv"

PATH_OUT = os.path.join(
    os.path.abspath(os.getcwd()),
    "data/_1_train_pump_sensor.csv"
)

# Load Cleaned Dataset

In [3]:
df_data = pd.read_csv(PATH_DATA)

print(df_data.shape)

df_data

(220320, 51)


,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,...,sensor_41,sensor_42,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,machine_status
0,2018-04-01 00:00:00,2.465394,47.09201,53.211800,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,...,30.989580,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,NORMAL
1,2018-04-01 00:01:00,2.465394,47.09201,53.211800,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,...,30.989580,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,NORMAL
2,2018-04-01 00:02:00,2.444734,47.35243,53.211800,46.397570,638.888900,73.54598,13.32465,16.03733,15.61777,...,30.468750,31.770830,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,NORMAL
3,2018-04-01 00:03:00,2.460474,47.09201,53.168400,46.397568,628.125000,76.98898,13.31742,16.24711,15.69734,...,30.468750,31.510420,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,NORMAL
4,2018-04-01 00:04:00,2.445718,47.13541,53.211800,46.397568,636.458300,76.58897,13.35359,16.21094,15.69734,...,30.989580,31.510420,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,NORMAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220315,2018-08-31 23:55:00,2.407350,47.69965,50.520830,43.142361,634.722229,64.59095,15.11863,16.65220,15.65393,...,30.468750,30.208330,38.28125,68.287030,52.37268,48.32176,41.087960,212.3843,153.64580,NORMAL
220316,2018-08-31 23:56:00,2.400463,47.69965,50.564240,43.142361,630.902771,65.83363,15.15480,16.70284,15.65393,...,30.208332,29.947920,38.28125,66.840280,50.63657,48.03241,40.798610,213.8310,156.25000,NORMAL
220317,2018-08-31 23:57:00,2.396528,47.69965,50.520830,43.142361,625.925903,67.29445,15.08970,16.70284,15.69734,...,29.947920,30.208330,39.06250,65.393520,48.90046,48.03241,40.798610,217.3032,155.38190,NORMAL
220318,2018-08-31 23:58:00,2.406366,47.69965,50.520832,43.142361,635.648100,65.09175,15.11863,16.56539,15.74074,...,29.947916,30.208332,40.62500,64.236110,47.74306,48.32176,40.509258,222.5116,153.93520,NORMAL


In [4]:
df_data['datetime'] = pd.to_datetime(df_data['timestamp'])
df_data.sort_values(by='datetime', ascending=True)

df_data

,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,...,sensor_42,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,machine_status,datetime
0,2018-04-01 00:00:00,2.465394,47.09201,53.211800,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,...,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,NORMAL,2018-04-01 00:00:00
1,2018-04-01 00:01:00,2.465394,47.09201,53.211800,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,...,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,NORMAL,2018-04-01 00:01:00
2,2018-04-01 00:02:00,2.444734,47.35243,53.211800,46.397570,638.888900,73.54598,13.32465,16.03733,15.61777,...,31.770830,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,NORMAL,2018-04-01 00:02:00
3,2018-04-01 00:03:00,2.460474,47.09201,53.168400,46.397568,628.125000,76.98898,13.31742,16.24711,15.69734,...,31.510420,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,NORMAL,2018-04-01 00:03:00
4,2018-04-01 00:04:00,2.445718,47.13541,53.211800,46.397568,636.458300,76.58897,13.35359,16.21094,15.69734,...,31.510420,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,NORMAL,2018-04-01 00:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220315,2018-08-31 23:55:00,2.407350,47.69965,50.520830,43.142361,634.722229,64.59095,15.11863,16.65220,15.65393,...,30.208330,38.28125,68.287030,52.37268,48.32176,41.087960,212.3843,153.64580,NORMAL,2018-08-31 23:55:00
220316,2018-08-31 23:56:00,2.400463,47.69965,50.564240,43.142361,630.902771,65.83363,15.15480,16.70284,15.65393,...,29.947920,38.28125,66.840280,50.63657,48.03241,40.798610,213.8310,156.25000,NORMAL,2018-08-31 23:56:00
220317,2018-08-31 23:57:00,2.396528,47.69965,50.520830,43.142361,625.925903,67.29445,15.08970,16.70284,15.69734,...,30.208330,39.06250,65.393520,48.90046,48.03241,40.798610,217.3032,155.38190,NORMAL,2018-08-31 23:57:00
220318,2018-08-31 23:58:00,2.406366,47.69965,50.520832,43.142361,635.648100,65.09175,15.11863,16.56539,15.74074,...,30.208332,40.62500,64.236110,47.74306,48.32176,40.509258,222.5116,153.93520,NORMAL,2018-08-31 23:58:00


In [5]:
df_data['machine_status'].value_counts()

NORMAL        205836
RECOVERING     14477
BROKEN             7
Name: machine_status, dtype: int64

# Label rows as per machine status

Mark all rows with status 'NORMAL' & 'RECOVERING' as 0 and 'BROKEN' as 1 in a new column.

In [6]:
MAP_LABEL = {
    'NORMAL': 0,
    'RECOVERING': 0,
    'BROKEN': 1
}

df_data['y'] = df_data['machine_status'].apply(lambda v: MAP_LABEL[v])

df_data

,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,...,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,machine_status,datetime,y
0,2018-04-01 00:00:00,2.465394,47.09201,53.211800,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,...,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,NORMAL,2018-04-01 00:00:00,0
1,2018-04-01 00:01:00,2.465394,47.09201,53.211800,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,...,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,NORMAL,2018-04-01 00:01:00,0
2,2018-04-01 00:02:00,2.444734,47.35243,53.211800,46.397570,638.888900,73.54598,13.32465,16.03733,15.61777,...,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,NORMAL,2018-04-01 00:02:00,0
3,2018-04-01 00:03:00,2.460474,47.09201,53.168400,46.397568,628.125000,76.98898,13.31742,16.24711,15.69734,...,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,NORMAL,2018-04-01 00:03:00,0
4,2018-04-01 00:04:00,2.445718,47.13541,53.211800,46.397568,636.458300,76.58897,13.35359,16.21094,15.69734,...,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,NORMAL,2018-04-01 00:04:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220315,2018-08-31 23:55:00,2.407350,47.69965,50.520830,43.142361,634.722229,64.59095,15.11863,16.65220,15.65393,...,38.28125,68.287030,52.37268,48.32176,41.087960,212.3843,153.64580,NORMAL,2018-08-31 23:55:00,0
220316,2018-08-31 23:56:00,2.400463,47.69965,50.564240,43.142361,630.902771,65.83363,15.15480,16.70284,15.65393,...,38.28125,66.840280,50.63657,48.03241,40.798610,213.8310,156.25000,NORMAL,2018-08-31 23:56:00,0
220317,2018-08-31 23:57:00,2.396528,47.69965,50.520830,43.142361,625.925903,67.29445,15.08970,16.70284,15.69734,...,39.06250,65.393520,48.90046,48.03241,40.798610,217.3032,155.38190,NORMAL,2018-08-31 23:57:00,0
220318,2018-08-31 23:58:00,2.406366,47.69965,50.520832,43.142361,635.648100,65.09175,15.11863,16.56539,15.74074,...,40.62500,64.236110,47.74306,48.32176,40.509258,222.5116,153.93520,NORMAL,2018-08-31 23:58:00,0


# Curve Shifting 

**10 minutes (10 Rows)**

1. For any row 'n' with label 1, make (n-10):(n-1) as 1. With this, we are teaching the classifier to predict up to 10 minutes ahead. 
2. Remove row 'n'. Row n is removed because we are not interested in teaching the classifier to predict a break when it has already happened.

In [7]:
TEST_IDXS = [17150, 17160]

df_data[TEST_IDXS[0]:TEST_IDXS[1]]

,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,...,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,machine_status,datetime,y
17150,2018-04-12 21:50:00,1.533738,52.03993,52.951385,43.402775,110.975121,46.92294,13.172740,16.65220,15.661170,...,49.47916,59.02778,50.92593,48.32176,42.245369,352.7199,111.9792,NORMAL,2018-04-12 21:50:00,0
17151,2018-04-12 21:51:00,1.349769,52.30035,52.951390,43.402770,110.512154,45.21725,13.201680,16.70284,15.892650,...,51.30208,56.71296,50.34722,49.76852,42.245370,361.4005,112.8472,NORMAL,2018-04-12 21:51:00,0
17152,2018-04-12 21:52:00,1.290741,52.51736,52.907990,43.402775,112.595482,47.36653,13.498260,16.65220,15.653930,...,52.08333,54.97685,49.47917,51.21528,41.956020,371.5278,113.4259,NORMAL,2018-04-12 21:52:00,0
17153,2018-04-12 21:53:00,1.220891,52.73437,52.821180,43.402775,113.174187,47.83727,13.252310,16.65220,15.733510,...,51.30208,53.24074,49.47917,52.08333,42.245369,384.5486,114.0046,NORMAL,2018-04-12 21:53:00,0
17154,2018-04-12 21:54:00,0.000000,52.95139,52.821178,43.402775,117.109375,48.17030,5.758102,16.70284,15.530960,...,51.04166,50.92593,50.34722,52.08333,42.534720,392.3611,115.4514,NORMAL,2018-04-12 21:54:00,0
17155,2018-04-12 21:55:00,0.000000,53.34201,52.821180,43.402775,202.526031,49.79289,3.219039,16.89091,16.869210,...,50.78125,50.92593,51.21528,50.63657,46.006940,409.1435,121.5278,BROKEN,2018-04-12 21:55:00,1
17156,2018-04-12 21:56:00,0.000000,53.55902,52.777770,43.402775,204.725098,53.74214,3.045428,17.42621,15.740740,...,49.47916,50.34722,51.21528,49.18982,49.479170,431.7130,133.1019,RECOVERING,2018-04-12 21:56:00,0
17157,2018-04-12 21:57:00,0.000000,53.55902,52.777770,43.402775,201.137131,52.49996,7.537616,13.53443,9.324364,...,48.17708,49.76852,50.92593,48.03241,53.819440,451.3889,140.0463,RECOVERING,2018-04-12 21:57:00,0
17158,2018-04-12 21:58:00,0.000000,52.77777,52.690970,43.402770,204.030655,57.19875,7.609953,16.60880,16.203700,...,46.87500,48.03241,50.34722,47.45370,58.159720,466.4352,144.9653,RECOVERING,2018-04-12 21:58:00,0
17159,2018-04-12 21:59:00,0.000000,52.60416,52.734370,43.446180,203.567688,50.96181,7.573785,16.70284,16.160300,...,45.83333,47.45370,49.76852,46.58565,63.657410,474.8264,149.0162,RECOVERING,2018-04-12 21:59:00,0


In [8]:
SHIFT_BY = 10


df_data_new = df_data.copy()

# (1) Shift values 
y_new = df_data_new['y'].to_list()
for idx, row in df_data_new.iterrows():
    if row['y'] == 1:
        _start_idx = idx-(SHIFT_BY)
        y_new[_start_idx:idx] = [1] * (idx-_start_idx)

df_data_new['label'] = y_new


# (2) Remove rows with status 'BROKEN'
df_data_new = df_data_new.drop(df_data_new[df_data_new['machine_status'] == 'BROKEN'].index)



df_data_new[TEST_IDXS[0]:TEST_IDXS[1]]

,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,...,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,machine_status,datetime,y,label
17150,2018-04-12 21:50:00,1.533738,52.03993,52.951385,43.402775,110.975121,46.92294,13.172740,16.65220,15.661170,...,59.02778,50.92593,48.32176,42.245369,352.7199,111.9792,NORMAL,2018-04-12 21:50:00,0,1
17151,2018-04-12 21:51:00,1.349769,52.30035,52.951390,43.402770,110.512154,45.21725,13.201680,16.70284,15.892650,...,56.71296,50.34722,49.76852,42.245370,361.4005,112.8472,NORMAL,2018-04-12 21:51:00,0,1
17152,2018-04-12 21:52:00,1.290741,52.51736,52.907990,43.402775,112.595482,47.36653,13.498260,16.65220,15.653930,...,54.97685,49.47917,51.21528,41.956020,371.5278,113.4259,NORMAL,2018-04-12 21:52:00,0,1
17153,2018-04-12 21:53:00,1.220891,52.73437,52.821180,43.402775,113.174187,47.83727,13.252310,16.65220,15.733510,...,53.24074,49.47917,52.08333,42.245369,384.5486,114.0046,NORMAL,2018-04-12 21:53:00,0,1
17154,2018-04-12 21:54:00,0.000000,52.95139,52.821178,43.402775,117.109375,48.17030,5.758102,16.70284,15.530960,...,50.92593,50.34722,52.08333,42.534720,392.3611,115.4514,NORMAL,2018-04-12 21:54:00,0,1
17156,2018-04-12 21:56:00,0.000000,53.55902,52.777770,43.402775,204.725098,53.74214,3.045428,17.42621,15.740740,...,50.34722,51.21528,49.18982,49.479170,431.7130,133.1019,RECOVERING,2018-04-12 21:56:00,0,0
17157,2018-04-12 21:57:00,0.000000,53.55902,52.777770,43.402775,201.137131,52.49996,7.537616,13.53443,9.324364,...,49.76852,50.92593,48.03241,53.819440,451.3889,140.0463,RECOVERING,2018-04-12 21:57:00,0,0
17158,2018-04-12 21:58:00,0.000000,52.77777,52.690970,43.402770,204.030655,57.19875,7.609953,16.60880,16.203700,...,48.03241,50.34722,47.45370,58.159720,466.4352,144.9653,RECOVERING,2018-04-12 21:58:00,0,0
17159,2018-04-12 21:59:00,0.000000,52.60416,52.734370,43.446180,203.567688,50.96181,7.573785,16.70284,16.160300,...,47.45370,49.76852,46.58565,63.657410,474.8264,149.0162,RECOVERING,2018-04-12 21:59:00,0,0
17160,2018-04-12 22:00:00,2.424498,52.51736,52.690970,43.446180,203.567688,49.78948,7.559317,16.56539,16.239870,...,46.29630,48.03241,45.13889,75.231480,477.7199,162.6157,RECOVERING,2018-04-12 22:00:00,0,0


In [9]:
df_data_new['label'].value_counts()

0    220243
1        70
Name: label, dtype: int64

# Save data

In [10]:
DIR_OUT = os.path.dirname(PATH_OUT)

if not os.path.exists(DIR_OUT):
    os.makedirs(DIR_OUT)

df_data_new.drop(columns=['datetime', 'y'], inplace=True)
df_data_new.to_csv(PATH_OUT, index=False)

print(f"Data saved to '{PATH_OUT}'.")

Data saved to '/home/gagan/Work/ai-ml-cv/anomaly-detection/pump_sensor/predict-breakpoint/data/_1_train_pump_sensor.csv'.
